In [1]:
import os
from pathlib import Path
import json
import glob
import h5py
import numpy as np
from tqdm.notebook import tqdm

In [2]:
outdir = '../../data_prods/'
data_dir = '/home/ryan/Data'

subdir_list = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

print(subdir_list)

['/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma0.1', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9', '/home/ryan/Data/run_2cDM_L3N512_HY_power00_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1', '/home/ryan/Data/run_2cDM_L25N256_DM_power00_sigma1_dir_4_Vkick55.65', '/home/ryan/Data/run_CDM_L5N256_DM', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_9', '/home/ryan/Data/run_SIDM_L3N256_DM_power0_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L25N256_DM_power00_sigma1_dir_7_Vkick119.90', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_1', '/home/ryan/Data/lost+found', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_3', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma10', '/home/ryan/Data/run_2

In [3]:
def make_CDF(data, nbins=1000):
    bins = np.geomspace(np.amin(data), np.amax(data), num=nbins)
    counts = np.array([ (data > b).sum() for b in bins ])
    
    return bins, counts

def get_profiles(f, nbins=1000):

    Subhalo=f.get('Subhalo')

    SFRs = Subhalo['SubhaloSFR'][()]
    SubhaloN = Subhalo['SubhaloLen'][()]
    enough = SubhaloN > 0

    sSFR = SFRs[enough]

    nonzero = sSFR[ sSFR != 0 ]
    if len(nonzero) > 0:
        SFR_bins, N_SFR = make_CDF(nonzero, nbins=nbins)
    else:
        SFR_bins = N_SFR = np.zeros(1, dtype=np.uint8)

    return (SFR_bins, N_SFR)

In [4]:
for subdir in tqdm(subdir_list):

    #run name is just folder name
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue

    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')
    print("Working on ", run_name)

    # read info from snapshots/fof
    snaps = sorted(glob.glob(subdir + '/snap*'))
    fofs = sorted(glob.glob(subdir + '/fof*'))

    if len(snaps) == 0:
        print(f'No snapshots for run {run_name}...')
        continue

    for fof in fofs:
        i = int(Path(fof).stem.split('_')[-1])
        with h5py.File(fof, 'r') as f:
            try:
                (sfr_bins, sfr_count) = get_profiles(f)
                tname = os.path.join(fpath, "SFR_profile_{}.txt".format(i))
                np.savetxt(tname, (sfr_bins, sfr_count))
            except KeyError:
                continue

  0%|          | 0/204 [00:00<?, ?it/s]

../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2 already exists!
Working on  run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2
../../data_prods/run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90 already exists!
Working on  run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1 already exists!
Working on  run_2cDM_L3N256_DM_powerm2m2_sigma0.1
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9 already exists!
Working on  run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1 already exists!
Working on  run_2cDM_L3N512_HY_power00_sigma1
../../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma0.1
../../data_prods/run_2cDM_L25N256_DM_power00_sigma1_dir_4_Vkick55.65 already exists!
Working on  run_2cDM_L25N256_DM_power00_sigma1_dir_4_Vkick55.65
../../data_prods/run_CDM_L5N256_DM already exists!
Working on  run_CDM_L5N256_DM
../../da